## Interfaces in Python
Every class has an interface: the set of public attributes (methods and data attributes) implemented or inherited by the class. Protocols are informal interfaces that make polymorphism work in languages with dynamic typing.

Interface might be defined as a subset of an object's public methods that enable it to play a specific role in the system. When we mention "an iterable" without specifying a class we mean it conforms to the iteration protocol.

An interface seen as a set of methods to fulfill a role is called a protocol.

Protocols might be partially implemented and cannot be enforced. Often "X-like object" "X protocol" and "X interface" are synonyms in the minds of Pythonistas.

## Sequence Protocol
This is one of the most fundamental protocols in Python and interpreter goes out of its way to handle objects that provide even a minimal implementation.
The philosophy of Python Data Model is to cooperate with essential protocols as much as possible.
Formal sequence protocol as defined by ABC supports:
`__getitem__`
`__contains__` inherited from Container
`__iter__` inherited from Iterable
`__reversed__`
index
count
`__len__` inherited from Sized

However even if we define `__getitem__` only we can get enough functionality for item access, iteration and in operation.

In [1]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

f = Foo()

In [2]:
print(f[1])

10


In [3]:
# There is no __iter__ method yet instances of Foo are iterable
for i in f:
    print(i)

0
10
20


In [4]:
print(20 in f)

True


In [5]:
print(15 in f)

False


Even in the absence of `__iter__` and `__contains__` Python interpreter will fall back on `__getitem__` and walk through the sequence to provide the missing functionality.

An iteration is an extreme form of duck typing the interpreter tries two different methods to iterate over an object.

## Monkey patching to implement a protocol at run time

In [6]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])
 
 
class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
     
    def __init__(self):
         self._cards = [Card(rank, suit) for suit in self.suits
                                         for rank in self.ranks]
         
    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

In [7]:
from random import shuffle
l = list(range(10))
shuffle(l)
l

[5, 4, 1, 7, 9, 0, 3, 2, 8, 6]

In [8]:
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [9]:
# Let us monkeypatch around this problem we can change the FrenchDeck class even at runtime

def set_card(deck, position, card):
    deck._cards[position] = card

FrenchDeck.__setitem__ = set_card

In [10]:
shuffle(deck)
deck[:5]

[Card(rank='8', suit='spades'),
 Card(rank='K', suit='hearts'),
 Card(rank='2', suit='clubs'),
 Card(rank='Q', suit='hearts'),
 Card(rank='K', suit='spades')]

This shows that every method starts its life just as a plain function and naming their arguments is just a convention. 
The signature for __setitem__ is described in language reference and normally it takes self, key, value


Monkey patching is about changing the module or class at runtime, without touching the source code. It is powerful but often leads to tightly coupled code as monkey patch need to know the internal workings of the object it patches, it can often handle undocumented and private parts.

All of these behaviours are examples of ducktyping operating with objects regardless of their types, as long as they implement certain protocols.